In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import defaultdict

import nltk
from nltk.corpus import brown
import math
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# Any results you write to the current directory are saved as output.
import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("../input"))

#scikit-learn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
import re
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.feature_selection import SelectKBest, chi2

import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print("Train shape : ", train_df.shape)
print("Test shape : ", test_df.shape)

In [ ]:
train_df["words#"] = train_df["question_text"].apply(lambda x: len(str(x).split()))
test_df["words#"] = test_df["question_text"].apply(lambda x: len(str(x).split()))

train_df["unique_words#"] = train_df["question_text"].apply(lambda x: len(set(str(x).split())))
test_df["unique_words#"] = test_df["question_text"].apply(lambda x: len(str(x).split()))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [ ]:
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [ ]:
from sklearn.preprocessing import FunctionTransformer
get_numeric_data = FunctionTransformer(lambda x: x[['words#','unique_words#']], validate=False)

classifier = Pipeline([
    ('features', FeatureUnion([
        ('text_features', Pipeline([
            ('colext', TextSelector('question_text')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     ngram_range=(1,3))),
            ('chi',  SelectKBest(chi2, k=500)),
            #('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
        ('numerical_features', Pipeline([
            ('wordext', get_numeric_data),
            ('wscaler', StandardScaler()),
        ])),
        
    ])),
  #  ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
   ('clf', RandomForestClassifier(class_weight='balanced_subsample')),
 # ('clf', LogisticRegression(C=5., solver='sag')),
    ])

In [ ]:
X = train_df[['question_text', 'words#','unique_words#']]
Y = train_df['target']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [ ]:
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

In [ ]:
score = model_selection.cross_val_score(classifier, X_test, y=y_test, cv=5, scoring='f1_macro')
score

In [ ]:
X_test_true = test_df[['question_text', 'words#','unique_words#']]
y_test_true = classifier.predict(X_test_true)
sub = pd.read_csv('../input/sample_submission.csv')
sub.prediction = y_test_true
sub.to_csv("submission.csv", index=False)